# KISSFLOW WEEKLY DISPLINARY ACTIONS REPORT

In [1]:
#Import Modules
import pandas as pd
from functools import reduce
from datetime import datetime
from datetime import timedelta 
import numpy as np
import os
os.chdir(r'C:\Users\user\Desktop\Analysis\KissFlow\Week of 2-9-2020')


In [10]:
#Import all items report data
AllItemData = pd.read_csv('Rwanda_Field_Team_disciplinary_actions.csv',
                          encoding='latin1',
                          usecols=['Case ID','Main case','First Name','Last Name','Region','District','Site','CreatedBy',
                                   'CreatedAt','Business_Step'])


In [11]:
#Import tasks report data
AllTasksData = pd.read_excel('AllTasks.xlsx') 

In [64]:
#Import cases under clarification
Clarify = pd.read_excel('Cases under clarification.xlsx')

### Cleaning task data

In [45]:
#Split Columns
SplitSubject = AllTasksData['Subject'].str.split("|", n = 5, expand = True)
SplitTaskName = AllTasksData['Task Name'].str.split("-", n = 2, expand = True)

#Change the date column of when a task has started to date type
AllTasksData['Started'] = AllTasksData['Started'].astype('datetime64[ns]')


#Assign split strings to data frame to create new columns
AllTasksData['Case ID']=SplitSubject[0]
AllTasksData['Case Type']=SplitSubject[1]
AllTasksData['Names']=SplitSubject[2]
AllTasksData['Payroll ID']=SplitSubject[3]
AllTasksData['Site']=SplitSubject[4]
AllTasksData['District']=SplitSubject[5]
AllTasksData['Department']=SplitTaskName[0]
AllTasksData['Task']=SplitTaskName[1]

#Correct departments naming to remove incorrect ones
AllTasksData['Department']=AllTasksData['Department'].replace(['Alice/Louise ','Confirmation of Legal Actions','FDV Manager assignement'],['Louise','Legal','FDV'])
AllTasksData['Task'] = AllTasksData['Task'].fillna('Confirmation of Legal Actions')
AllTasksData['Task'] = AllTasksData['Task'].str.strip()
AllTasksData['Department']= AllTasksData['Department'].str.strip()

#CLEANING CASES UNDER CLARIFICALION
SplitItemSubject = Clarify['Subject'].str.split("|", n = 5, expand = True)
Clarify['Case ID']=SplitItemSubject[0]
Clarify['Case Type']=SplitItemSubject[1]
Clarify['Names']=SplitItemSubject[2]
Clarify['Payroll ID']=SplitItemSubject[3]
Clarify['Site']=SplitItemSubject[4]
Clarify['District']=SplitItemSubject[5]

### Report on total, completed and pending tasks

In [46]:
#Completed and pending taks per department
TotalTasks = AllTasksData.groupby('Department')['Case ID'].count().reset_index().rename(columns={'Case ID': 'Total Tasks Assigned'})
PendingTasks = AllTasksData[AllTasksData['completed?']=='No']
PendingTasks = PendingTasks .groupby('Department')['Case ID'].count().reset_index().rename(columns={'Case ID': 'Total Pending Tasks'})
CompletedTasks = AllTasksData[AllTasksData['completed?']=='Yes']
CompletedTasks = CompletedTasks.groupby('Department')['Case ID'].count().reset_index().rename(columns={'Case ID': 'Total completed Tasks'})

#Merge total, completed and pending tasks and prepare their percentages
AllDataReport = TotalTasks.merge(CompletedTasks,on='Department',how='outer').merge(PendingTasks,on='Department',how='outer')
AllDataReport['Total Pending Tasks'] = AllDataReport['Total Pending Tasks'].fillna(0)
AllDataReport['% Completed tasks']=AllDataReport['Total completed Tasks']*100/AllDataReport['Total Tasks Assigned']
AllDataReport['% Pending tasks']=AllDataReport['Total Pending Tasks']*100/AllDataReport['Total Tasks Assigned']

#Final report
AllDataReport

Department  Total Tasks Assigned  Total completed Tasks  \
0         FDV                    35                     31   
1          HR                   145                    131   
2       Legal                     8                      4   
3      Louise                    43                     43   
4  Magnifique                    22                     21   
5          RL                    69                     47   

   Total Pending Tasks  % Completed tasks  % Pending tasks  
0                  4.0          88.571429        11.428571  
1                 14.0          90.344828         9.655172  
2                  4.0          50.000000        50.000000  
3                  0.0         100.000000         0.000000  
4                  1.0          95.454545         4.545455  
5                 22.0          68.115942        31.884058

### Report on Tasks and SLAs

In [55]:
#Build SLA dataframe
SLAs = pd.DataFrame({'Task':['Confirmation of employee benefits',
                    'Update Roster',
                    'Confirmation of Legal Actions',
                    'Manager assignment',
                    'Approval of the decision taken',
                    'Confirm receipt of signed letter from staff',
                    'Initial Follow up on the case',
                    'To process the case',
                    'confirm a new employee',
                    'Write off clients credit',
                    'Decision implementation/submit documents signed by staff',
                    'Inform clients about new employee',
                    'Upload explanation letter from absent employee',
                    'hiring a new employee',
                    'Provision of Investigation results'],'SLA':[2,2,2,2,2,2,2,7,17,2,2,2,2,10,3]})

#Prepare pending tasks only data frame
AllPendingTasks = AllTasksData[AllTasksData['completed?']=='No']

#Merge all pending tasks with SLA
TasksWithSLA = pd.merge(AllPendingTasks,SLAs, on='Task', how='left')

#Add a deadline column
TasksWithSLA['Deadline'] = TasksWithSLA['Started']+pd.to_timedelta(TasksWithSLA['SLA'], unit='d')
TasksWithSLA["Day"] = TasksWithSLA['Deadline'].dt.day_name()

#Add Days for tasks with deadline in the weekend
ConditionsForAddition = [(TasksWithSLA["Day"]=='Saturday'),((TasksWithSLA["Day"]=='Sunday'))]
ValuesForAddition = [2,2]
TasksWithSLA["Additional Days"]=np.select(ConditionsForAddition,ValuesForAddition)

#Final deadline
TasksWithSLA['Final Deadline'] = TasksWithSLA['Deadline']+pd.to_timedelta(TasksWithSLA['Additional Days'], unit='d')

In [57]:
Today = '2020-09-03'
SevereSLABreach = '2020-08-24'

In [61]:
#Prepare conditions for SLA status
conditions = [(TasksWithSLA['Final Deadline']>=Today),
              (TasksWithSLA['Final Deadline']<Today)&(TasksWithSLA['Final Deadline']>=SevereSLABreach),
              (TasksWithSLA['Final Deadline']<SevereSLABreach)]

#Values if a given condition is met
values = ['Within SLA','SLA Breach', 'Severe SLA Breach']

#Add conditions and values together
TasksWithSLA_NoWeekends['SLA Status'] = np.select(conditions, values)

#Reduced SLA status dataframe
SLAStatus = TasksWithSLA_NoWeekends[['Case ID','Department','SLA Status']]

#SLA status report
SLAStatusReport = pd.pivot_table(SLAStatus, values='Case ID', 
                                 index=['Department'],
                                 columns=['SLA Status'], 
                                 aggfunc='count',
                                fill_value=0,
                                margins=True).reset_index().rename(columns={'All':'Total cases per owner'})

#Add correct naming for totals
SLAStatusReport['Department']=SLAStatusReport['Department'].replace(['All'],'Total cases per SLA Status')

#Final report
SLAStatusReport

SLA Status                  Department  SLA Breach  Severe SLA Breach  \
0                                 FDV            1                  2   
1                                  HR            3                  1   
2                                Legal           4                  0   
3                          Magnifique            1                  0   
4                                  RL            9                 10   
5           Total cases per SLA Status          18                 13   

SLA Status  Within SLA  Total cases per owner  
0                    1                      4  
1                   10                     14  
2                    0                      4  
3                    0                      1  
4                    3                     22  
5                   14                     45

#### Cases by type and region

In [62]:
#Add correct naming for department names
AllItemData['Main case']=AllItemData['Main case'].replace(['Performance'],['Poor Performance'])

#Summarize with pivot table cases by type and region
TypeAndRegionReport = pd.pivot_table(AllItemData, values='Case ID', 
                                     index=['Main case'],
                                     columns=['Region'], 
                                     aggfunc='count',
                                     margins=True).fillna(0).reset_index().rename(columns={'All':'Tota cases per case type',
                                                                                                           'Main case':'Case Type'})
#Add correct naming for totals column
TypeAndRegionReport['Case Type']=TypeAndRegionReport['Case Type'].replace(['All'],'Total case per Region')

#Final report
TypeAndRegionReport

Region              Case Type  East  South  South West  West  \
0                 Absenteeism   0.0    1.0         1.0   0.0   
1                       Fraud  15.0   16.0         5.0  11.0   
2            Poor Performance   2.0    1.0         0.0   1.0   
3                 Resignation   9.0    3.0         1.0   3.0   
4       Total case per Region  26.0   21.0         7.0  15.0   

Region  Tota cases per case type  
0                              2  
1                             47  
2                              4  
3                             16  
4                             69